# A3C for Kung Fu

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [14]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importing the libraries

In [15]:
import cv2
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torch.multiprocessing as mp 
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [16]:
class Network(nn.Module):
    def __init__(self,action_size):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, 3,2)
        self.conv2 = nn.Conv2d(32, 32, 3,2)
        self.conv3 = nn.Conv2d(32, 32, 3,2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, 128)
        self.fc2a = nn.Linear(128, action_size)
        self.fc2s = nn.Linear(128, 1)
    
    def forward(self,state):
        x = F.relu(self.conv1(state))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        action_values = self.fc2a(x)
        static_value = self.fc2s(x)[0]
        return action_values, static_value


## Part 2 - Training the AI

### Setting up the environment

In [17]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4):
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.dim_order = dim_order
    self.color = color
    self.frame_stack = n_frames
    n_channels = 3 * n_frames if color else n_frames
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]
    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.frames = np.zeros(obs_shape, dtype = np.float32)

  def reset(self):
    self.frames = np.zeros_like(self.frames)
    obs, info = self.env.reset()
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype('float32') / 255.
    if self.color:
      self.frames = np.roll(self.frames, shift = -3, axis = 0)
    else:
      self.frames = np.roll(self.frames, shift = -1, axis = 0)
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img
    return self.frames

  def update_buffer(self, obs):
    self.frames = self.observation(obs)

def make_env():
  env = gym.make("MsPacmanDeterministic-v0", render_mode = 'rgb_array')
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  return env

env = make_env()

state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("State shape:", state_shape)
print("Number actions:", number_actions)
print("Action names:", env.env.env.get_action_meanings())

State shape: (4, 42, 42)
Number actions: 9
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT']


C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.get_action_meanings to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_action_meanings` for environment variables or `env.get_wrapper_attr('get_action_meanings')` that will search the reminding wrappers.
  logger.warn(


### Initializing the hyperparameters

In [18]:
learning_rate = 1e-4
discount_factor = 0.99
number_environments = 10

### Implementing the A3C class

In [19]:
class Agent():
    def __init__(self, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.action_size = action_size
        self.network = Network(action_size).to(self.device)
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr= learning_rate)
    
    def act(self, state):
        if state.ndim == 3:
            state = [state]
        state = torch.tensor(state, dtype= torch.float32, device= self.device)
        action_values, _ = self.network(state) # == self.network.forward(state) pytorch knows the forward method alone
        policy = F.softmax(action_values, dim= -1)
        return np.array([np.random.choice(len(p), p = p) for p in policy.detach().cpu().numpy()])
    
    def step(self, state, action, reward, next_state, done):
        batch_size = state.shape[0]
        # casting types to all float
        state = torch.tensor(state, dtype= torch.float32, device= self.device)
        next_state = torch.tensor(next_state, dtype= torch.float32, device = self.device)
        reward = torch.tensor(reward, dtype= torch.float32, device= self.device)
        done = torch.tensor(done, dtype= torch.bool, device= self.device).to(dtype= torch.float32)
        # get the predicted values
        action_values, state_value = self.network(state)
        _, next_state_value = self.network(next_state)
        # target value
        target_state_value = reward + discount_factor * next_state_value * (1 - done)
        # calculate the losses
        advantage = target_state_value - state_value
        probs = F.softmax(action_values, dim= -1)
        logprobs = F.log_softmax(action_values, dim=-1)
        entropy = - torch.sum(probs * logprobs, axis = -1)
        batch_idx = np.arange(batch_size)
        logp_actions = logprobs[batch_idx, action]
        actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()
        critic_loss = F.mse_loss(target_state_value.detach(), state_value)
        total_loss = actor_loss + critic_loss
        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

### Initializing the A3C agent

In [20]:
agent = Agent(number_actions)

### Evaluating our A3C agent on a certain number of episodes

In [21]:
def eval(agent, env, n_episodes):
    episodes_rewards = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_rewards = 0
        while True:
            action = agent.act(state)
            state, reward, done, info, _ = env.step(action[0])
            total_rewards += reward
            if done:
                break 
        episodes_rewards.append(total_rewards)
    return episodes_rewards

### Managing multiple environments simultaneously

In [22]:
class EnvBatch():
    def __init__(self, n_env = 10):
        self.envs = [make_env() for _ in range(n_env)]
    
    def reset(self):
        _states = []
        for env in self.envs:
            _states.append(env.reset()[0])
        return np.array(_states)
    def step(self, actions):
        next_states, rewards, dones, info, _ = map(np.array, zip(*[env.step(a) for env,a in zip(self.envs, actions)]))
        for i in range(len(self.envs)):
            if dones[i]:
                next_states[i] = self.envs[i].reset()[0]
        return next_states, rewards, dones, info

### Training the A3C agent

In [23]:
import tqdm # for progress bars

env_batch = EnvBatch(number_actions)
batch_states = env_batch.reset()
with tqdm.trange(0, 10001) as progress_bar:
    for i in progress_bar:
        batch_actions = agent.act(batch_states)
        batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions)
        batch_rewards *= 0.01
        agent.step(batch_states,batch_actions, batch_rewards, batch_next_states, batch_dones)
        batch_states = batch_next_states
        if i % 1000 == 0:
            print("Average agent reward: ", np.mean(eval(agent,env, n_episodes=10)))

  0%|          | 0/10001 [00:00<?, ?it/s]C:\Users\Hima\AppData\Local\Temp\ipykernel_13036\4120483982.py:36: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([9])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(target_state_value.detach(), state_value)
  0%|          | 10/10001 [00:11<2:22:45,  1.17it/s]

Average agent reward:  253.0


 10%|█         | 1009/10001 [00:36<1:01:06,  2.45it/s]

Average agent reward:  249.0


 20%|██        | 2009/10001 [01:01<52:42,  2.53it/s]  

...

Average agent reward:  393.0


 90%|█████████ | 9010/10001 [03:51<05:42,  2.90it/s]

Average agent reward:  386.0


100%|██████████| 10001/10001 [04:15<00:00, 39.11it/s]

Average agent reward:  427.0


## Part 3 - Visualizing the results

In [27]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env):
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()